In [2]:
class AlgorithmState():
    def __init__(self, m, t, output = True):
        self.m = m
        self.t = t
        self.output = output
        
    def apply_transform(self, transform):
        raise NotImplementedError("The method apply_transform has not been defined on this state type")
        
    def compose_transform(self, t1, t2):
        raise NotImplementedError("The method compose_transform has not been defined on this state type")
        
    def step(self, transform, description = None):
        m_before = self.m
        self.apply_transform(transform)
        if self.m == m_before:
            return
        self.t = self.compose_transform(self.t, transform)
        if self.output:
            if description is not None:
                print(description)
            pretty_print(self.m)
        
    def row_addition_transform(self, source, target, scale = 1):
        raise NotImplementedError("The method row_addition_transform has not been defined on this state type")
        
    def add_row(self, source, target, scale = 1, description = None):
        self.step(self.row_addition_transform(source, target, scale), description)
        
    def switch_rows(self, source, target, description = None):
        self.step(
            self.compose_transform(
                self.compose_transform(
                    self.row_addition_transform(source, target),
                    self.row_addition_transform(target, source, scale = -1),
                ),
                self.row_addition_transform(source, target),
            ), description
        )
    
    def column_addition_transform(self, source, target, scale = 1):
        raise NotImplementedError("The method column_addition_transform has not been defined on this state type")
    
    def add_column(self, source, target, scale = 1, description = None):
        self.step(self.column_addition_transform(source, target, scale), description)
        
    def switch_columns(self, source, target, description = None):
        self.step(
            self.compose_transform(
                self.compose_transform(
                    self.column_addition_transform(source, target),
                    self.column_addition_transform(target, source, scale = -1),
                ),
                self.column_addition_transform(source, target),
            ), description
        )

In [3]:
class SLnState(AlgorithmState):
    def __init__(self, m, output = True):
        dim = m.nrows()
        super().__init__(m, (identity_matrix(dim), identity_matrix(dim)), output)
        self.dim = dim
        
    def apply_transform(self, transform):
        self.m = transform[0] * self.m * transform[1]
        
    def compose_transform(self, t1, t2):
        return (t2[0] * t1[0], t1[1] * t2[1])
        
    def column_addition_transform(self, source, target, scale = 1):
        return (identity_matrix(self.dim), elementary_matrix(ZZ, self.dim, row1 = source, row2 = target, scale = scale))

    def row_addition_transform(self, source, target, scale = 1):
        return (elementary_matrix(ZZ, self.dim, row1 = target, row2 = source, scale = scale), identity_matrix(self.dim))

In [4]:
class SL2SL2State(AlgorithmState):
    def __init__(self, m, output = True):
        super().__init__(m, (identity_matrix(2), identity_matrix(2)), output)
        
    def project(self, m1, m2):
        return matrix([[m1[y // 2, x // 2] * m2[y % 2, x % 2] for x in range(4)] for y in range(4)])
    
    def apply_transform(self, transform):
        self.m = self.project(transform[0], transform[1]) * self.m
    
    def compose_transform(self, t1, t2):
        return (t2[0] * t1[0], t2[1] * t1[1])
    
    def row_addition_transform(self, source, target, scale = 1):
        offset = min(source, target)
        source -= offset
        target -= offset
        if source == 0 and target == 1:
            return (identity_matrix(2), elementary_matrix(ZZ, 2, row1 = 1, row2 = 0, scale = scale))
        if source == 1 and target == 0:
            return (identity_matrix(2), elementary_matrix(ZZ, 2, row1 = 0, row2 = 1, scale = scale))
        if source == 0 and target == 2:
            return (elementary_matrix(ZZ, 2, row1 = 1, row2 = 0, scale = scale), identity_matrix(2))
        if source == 2 and target == 0:
            return (elementary_matrix(ZZ, 2, row1 = 0, row2 = 1, scale = scale), identity_matrix(2))
    
    def column_addition_transform(self, source, target, scale = 1):
        raise NotImplementedError("Column manipulation is not implemented")

In [5]:
class ModelState(AlgorithmState):
    def __init__(self, m, output = True):
        dim = m.nrows()
        super().__init__(m, identity_matrix(dim), output)
        self.dim = dim
    
    def apply_transform(self, transform):
        self.m = transform.transpose() * self.m * transform
        
    def compose_transform(self, t1, t2):
        return t1 * t2
    
    def row_addition_transform(self, source, target, scale = 1):
        return elementary_matrix(scale.parent(), self.dim, row1 = source, row2 = target, scale = scale)
    
    def column_addition_transform(self, source, target, scale = 1):
        return self.row_addition_transform(source, target, scale)

In [ ]:
class ModelImprovementState(AlgorithmState):
    def __init__(self, m, improvement, output = True):
        dim = m.nrows()
        super().__init__((m, improvement), (identity_matrix(dim), identity_matrix(dim)), output)
        self.dim = dim
    
    @property
    def improvement(self):
        return self.m[1]
    
    def apply_transform(self, transform):
        (m1, m2) = self.m
        (t1, t2) = transform
        self.m = (t1.inverse().transpose() * m1 * t1.inverse(), t1 * m2 * t2)
        
    def compose_transform(self, t1, t2):
        return (t2[0] * t1[0], t1[1] * t2[1])
    
    def column_addition_transform(self, source, target, scale = 1):
        return (identity_matrix(self.dim), elementary_matrix(scale.parent(), self.dim, row1 = source, row2 = target, scale = scale))

    def row_addition_transform(self, source, target, scale = 1):
        return (elementary_matrix(scale.parent(), self.dim, row1 = target, row2 = source, scale = scale), identity_matrix(self.dim))